# 패키지 불러오기

In [1]:
!openai --version

openai 1.35.7


In [3]:
import os
import time
import yaml
import re
import json

from openai import OpenAI
import pandas as pd
import ipywidgets as widgets # interactive display
from tqdm import tqdm # progress bar
from dotenv import load_dotenv

# ENV, Config 파일 읽기

In [4]:
# .env 파일에서 환경 변수 로드(API 키)
load_dotenv()

True

In [11]:
# YAML 파일 열기
yaml_path = 'config.yaml' # todo: config 파일과 합치기

with open(yaml_path, 'r') as f:
    config = yaml.safe_load(f)

# 인스턴스 생성

In [6]:
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# 함수 정의

In [7]:
def get_response(messages):
    """
    model: 모델 종류
    messages: 사용자의 입력과 모델의 출력을 교환하는 메시지 목록
    max_tokens: 생성될 응답의 최대 길이
    temperature: 생성될 응답의 다양성(0.0 ~ 1.0) 0.0은 가장 확실한 답변을, 1.0은 가장 다양한 답변을 생성
    stream: 응답을 한 번에 반환할지 여부. False로 설정하면 한 번에 반환
    """
    response = client.chat.completions.create(
        model = "gpt-4o",
        messages = messages,
        # max_tokens = 150,
        temperature = 0.0,
        stream = False)
    return response.choices[0].message.content

In [18]:
def extract_issues(content):
    # issue 부분을 정규 표현식을 사용하여 추출
    issues_match = re.search(r"\[issue\](.*?)\[sentiment\]", content, re.DOTALL)
    if issues_match:
        issues_str = issues_match.group(1).strip()
        issues = [issue.strip() for issue in issues_str.split(',')]
        return issues
    return []

def extract_sentiments(content):
    # sentiment 부분을 정규 표현식을 사용하여 추출
    sentiments_match = re.search(r"\[sentiment\](.*?)\[sentiment\(all\)\]", content, re.DOTALL)
    if sentiments_match:
        sentiments_str = sentiments_match.group(1).strip()
        sentiments = [sentiment.strip() for sentiment in sentiments_str.split(',')]
        return sentiments
    return []

def extract_sentiment_all(content):
    # sentiment(all) 부분을 정규 표현식을 사용하여 추출
    sentiment_all_match = re.search(r"\[sentiment\(all\)\](.*)", content, re.DOTALL)
    if sentiment_all_match:
        sentiment_all = sentiment_all_match.group(1).strip()
        return sentiment_all
    return ""

# 파일 읽기

# test 영역

In [15]:
instructions = config['gptapi']['sentiment']['instructions']
few_shot_examples = config['gptapi']['sentiment']['few_shot_examples']

In [8]:
# 처리할 데이터
df = pd.read_csv('data/제공/insight_Paprika Story_20240603.csv')
df.head()

,date,tid,raw_text,theme,issue,mention,sentiment,rating_score,product,data_origin,contact_name
0,2024-05-27 13:05:13,FB-830,잘대응해주시었습니다~^^수고하세요,고객 서비스,NaN,잘대응해주시었습니다수고하세요.,VERY_POSITIVE,5,스탠바이미고,코와샵,ash_****
1,2024-05-27 13:05:01,FB-693,빠른배송 맘에 듭니다. 기사님이 친절하세요.,고객 서비스,친절한 판매자 및 설치 기사,기사님이 친절하세요.,VERY_POSITIVE,5,스탠바이미고,롯데ON,e*l*****
2,2024-05-27 13:04:55,FB-535,제품좋고 배송도 빠르고 친절하시고 너무 좋습니다. 다만 제가 구매확정 늦게 처리해서...,고객 서비스,친절한 판매자 및 설치 기사,친절하시고,VERY_POSITIVE,5,스탠바이미고,옥션,c*o*****
3,2024-05-27 13:04:55,FB-563,빠른 배송과 아울러 문의를 여러 번 했는데 친절하게 답변해주셔서 감사합니다! 매우 ...,고객 서비스,빠르고 친절한 응답,문의를 여러 번 했는데 친절하게 답변해주셔서 감사합니다,VERY_POSITIVE,5,스탠바이미고,옥션,s*i*****
4,2024-05-27 13:05:01,FB-685,"판매자분도 친절하시고, 기사분도 친절하셔서 여러모로 만족해요! 제품도 좋네요 굿굿",고객 서비스,일반적인 서비스 만족도,"판매자분도 친절하시고, 기사분도 친절하셔서 여러모로 만족해요!",VERY_POSITIVE,5,스탠바이미고,옥션,d*c*****


In [17]:
test = df['raw_text'][:10]
print(test)

0                                   잘대응해주시었습니다~^^수고하세요
1                             빠른배송 맘에 듭니다. 기사님이 친절하세요.
2    제품좋고 배송도 빠르고 친절하시고 너무 좋습니다. 다만 제가 구매확정 늦게 처리해서...
3    빠른 배송과 아울러 문의를 여러 번 했는데 친절하게 답변해주셔서 감사합니다! 매우 ...
4        판매자분도 친절하시고, 기사분도 친절하셔서 여러모로 만족해요! 제품도 좋네요 굿굿
5                                 품질좋고 기사님아주 친절하십니다^^^
6                    문의드리면 답변도 빠르시고 물건도하자없이 왔어요 감사합니다!
7    주문한지 40일만에 물건받았습니다. 중간에 취소하고 다른 곳에서 주문할까 했지만 제...
8    판매자님 상담도 친절하게 해주셨는데 배송도 바로 해주셔서 주문하고 다음날인가? 하루...
9                                        배송도 빠르고 친절합니다
Name: raw_text, dtype: object


In [16]:
# 결과를 저장할 리스트
raw_texts_list = []
issues_list = []
sentiments_list = []
sentiment_all_list = []

total_iterations = len(test)

with tqdm(total=total_iterations) as pbar:
    for i, text in enumerate(test):

        # (Step 1) 각 raw_text에 대해 issue와 sentiment 추출
        # prompt 생성
        raw_text = text
        system_message = {"role": "system", "content": instructions}
        user_message = {"role": "user", "content": f"""{few_shot_examples}\n[raw_text]\n{raw_text}"""}
        messages = [system_message, user_message]

        # GPT-4o로부터 응답 받기
        content = get_response(messages)
        
        # 응답 파싱
        issue = extract_issues(content)
        sentiment = extract_sentiments(content)
        sentiment_all = extract_sentiment_all(content)
        
        # 결과 저장
        issues_list.append(issue)
        sentiments_list.append(sentiment)
        sentiment_all_list.append(sentiment_all)
        
        # (Step 2) 모든 issue 데이터를 기반으로 theme 추출 
        # 모든 issue를 리스트로 합침
        all_issues = [item for sublist in issues_list for item in sublist]
        
        
        pbar.update(1)
        time.sleep(1)



(1446, 11)

In [ ]:
results_list = []
# 테스트 데이터: test
total_iterations = len(test)

with tqdm(total=total_iterations, desc="Processing") as pbar:
    for i in test:
        messages = [
            {"role": "system", "content": config['gptapi']['system_content']['role_summarize']},
            {"role": "user", "content": i},
        ]
        # 요약 결과를 받아오기
        response_sum = get_response(messages)
        messages.append({"role": "assistant", "content": response_sum})
        # 감정 분석 결과를 받아오기
        input_senti = config['gptapi']['assistant_content']['senti_categ']
        messages.append({"role": "user", "content": input_senti})
        response_senti = get_response(messages)

        results_list.append({'original_message': i, 'summary': response_sum, 'sentiment': response_senti})
        pbar.update(1)
        time.sleep(1)

results = pd.DataFrame(results_list)

Processing: 100%|██████████| 10/10 [00:33<00:00,  3.36s/it]
